In [1]:
from IPython.display import YouTubeVideo
import numpy as np
import pandas as pd
!pip install pyexcel
import pyexcel as pe
!pip install pyexcel-xlsx
from openpyxl import load_workbook

In [2]:
#tutorial on YouTube
YouTubeVideo('T3UZUftD4-Y')

# **pandas Module**

In [3]:
df = pd.read_csv('/content/Financial_data_combined.csv')

In [4]:
df.head(10)

,Date,S&P 500 Open,S&P 500 High,S&P 500 Low,S&P 500 Close,NASDAQ Open,NASDAQ High,NASDAQ Low,NASDAQ Close
0,8/20/2021,14571.53,14722.16,14571.53,14714.66,4410.56,4444.35,4406.80,4441.67
1,8/19/2021,14423.16,14610.76,14423.16,14541.79,4382.44,4418.61,4367.73,4405.80
2,8/18/2021,14636.24,14697.25,14516.30,14525.91,4440.94,4454.32,4397.59,4400.27
3,8/17/2021,14670.56,14716.95,14550.88,14656.18,4462.12,4462.12,4417.83,4448.08
4,8/16/2021,14771.53,14794.68,14610.02,14793.76,4461.65,4480.26,4437.66,4479.71
5,8/13/2021,14825.07,14850.61,14797.22,14822.90,4464.84,4468.37,4460.82,4468.00
6,8/12/2021,14751.36,14824.31,14698.94,14816.26,4446.08,4461.77,4435.96,4460.83
7,8/11/2021,14834.38,14842.50,14692.41,14765.14,4442.18,4449.44,4436.42,4447.70
8,8/10/2021,14887.75,14894.61,14754.48,14788.09,4435.79,4445.21,4430.03,4436.75
9,8/9/2021,14855.76,14883.81,14802.69,14860.18,4437.77,4439.39,4424.74,4432.35


In [5]:
df.dtypes

Date              object
S&P 500 Open     float64
S&P 500 High     float64
S&P 500 Low      float64
S&P 500 Close    float64
NASDAQ Open      float64
NASDAQ High      float64
NASDAQ Low       float64
NASDAQ Close     float64
dtype: object

In [6]:
df.loc[:,df.columns[1:]].apply(pd.to_numeric)

,S&P 500 Open,S&P 500 High,S&P 500 Low,S&P 500 Close,NASDAQ Open,NASDAQ High,NASDAQ Low,NASDAQ Close
0,14571.53,14722.16,14571.53,14714.66,4410.56,4444.35,4406.80,4441.67
1,14423.16,14610.76,14423.16,14541.79,4382.44,4418.61,4367.73,4405.80
2,14636.24,14697.25,14516.30,14525.91,4440.94,4454.32,4397.59,4400.27
3,14670.56,14716.95,14550.88,14656.18,4462.12,4462.12,4417.83,4448.08
4,14771.53,14794.68,14610.02,14793.76,4461.65,4480.26,4437.66,4479.71
...,...,...,...,...,...,...,...,...
248,11516.62,11672.05,11507.46,11665.06,3449.97,3481.07,3444.15,3478.73
249,11370.23,11468.26,11343.04,11466.47,3435.95,3444.21,3425.84,3443.62
250,11449.25,11462.05,11297.53,11379.72,3418.09,3432.09,3413.13,3431.28
251,11258.45,11326.22,11245.45,11311.80,3386.01,3399.96,3379.31,3397.16


In [7]:
df['NASDAQ Log Return'] = np.log(df['NASDAQ Close']) - np.log(df['NASDAQ Close'].shift(-1))
df['S&P 500 Log Return'] = np.log(df['S&P 500 Close']) - np.log(df['S&P 500 Close'].shift(-1))

In [8]:
df

,Date,S&P 500 Open,S&P 500 High,S&P 500 Low,S&P 500 Close,NASDAQ Open,NASDAQ High,NASDAQ Low,NASDAQ Close,NASDAQ Log Return,S&P 500 Log Return
0,8/20/2021,14571.53,14722.16,14571.53,14714.66,4410.56,4444.35,4406.80,4441.67,0.008109,0.011818
1,8/19/2021,14423.16,14610.76,14423.16,14541.79,4382.44,4418.61,4367.73,4405.80,0.001256,0.001093
2,8/18/2021,14636.24,14697.25,14516.30,14525.91,4440.94,4454.32,4397.59,4400.27,-0.010807,-0.008928
3,8/17/2021,14670.56,14716.95,14550.88,14656.18,4462.12,4462.12,4417.83,4448.08,-0.007086,-0.009343
4,8/16/2021,14771.53,14794.68,14610.02,14793.76,4461.65,4480.26,4437.66,4479.71,0.002617,-0.001968
...,...,...,...,...,...,...,...,...,...,...,...
248,8/26/2020,11516.62,11672.05,11507.46,11665.06,3449.97,3481.07,3444.15,3478.73,0.010144,0.017171
249,8/25/2020,11370.23,11468.26,11343.04,11466.47,3435.95,3444.21,3425.84,3443.62,0.003590,0.007594
250,8/24/2020,11449.25,11462.05,11297.53,11379.72,3418.09,3432.09,3413.13,3431.28,0.009994,0.005986
251,8/21/2020,11258.45,11326.22,11245.45,11311.80,3386.01,3399.96,3379.31,3397.16,0.003435,0.004150


In [9]:
sandp = df.loc[:,df.columns[1:5]]

In [10]:
nasdaq = df.drop(columns=df.columns[1:5])
nasdaq = nasdaq.loc[:,nasdaq.columns[:-2]]

In [11]:
writer = pd.ExcelWriter('/content/Financial_data.xlsx')

sandp.to_excel(writer, sheet_name='S&P500', index = False)
nasdaq.to_excel(writer, sheet_name='Nasdaq', index = False)

writer.save()

# **pyexcel Module**

In [12]:
fin_data = pe.get_book(file_name = '/content/Financial_data.xlsx')

In [13]:
fin_data

S&P500:
+--------------+--------------+-------------+---------------+
| S&P 500 Open | S&P 500 High | S&P 500 Low | S&P 500 Close |
+--------------+--------------+-------------+---------------+
| 14571.53     | 14722.16     | 14571.53    | 14714.66      |
+--------------+--------------+-------------+---------------+
| 14423.16     | 14610.76     | 14423.16    | 14541.79      |
+--------------+--------------+-------------+---------------+
| 14636.24     | 14697.25     | 14516.3     | 14525.91      |
+--------------+--------------+-------------+---------------+
| 14670.56     | 14716.95     | 14550.88    | 14656.18      |
+--------------+--------------+-------------+---------------+
| 14771.53     | 14794.68     | 14610.02    | 14793.76      |
+--------------+--------------+-------------+---------------+
| 14825.07     | 14850.61     | 14797.22    | 14822.9       |
+--------------+--------------+-------------+---------------+
| 14751.36     | 14824.31     | 14698.94    | 14816.26      |


In [14]:
fin_dict = fin_data.dict

In [15]:
pct_chnge = []

for row,row_lag in zip(fin_dict['S&P500'][1:],fin_dict['S&P500'][2:]):
  pct_chnge.append((np.log(row[-1]/row_lag[-1])))

In [16]:
for row,rtn in zip(fin_dict['S&P500'][1:], pct_chnge):
  row.append(rtn)

fin_dict['S&P500'][0].append('Log Return')

In [17]:
pe.save_as(array = fin_dict['S&P500'], dest_file_name = '/content/s&p_data.xlsx')

In [18]:
pd.read_excel('/content/s&p_data.xlsx')

,S&P 500 Open,S&P 500 High,S&P 500 Low,S&P 500 Close,Log Return
0,14571.53,14722.16,14571.53,14714.66,0.011818
1,14423.16,14610.76,14423.16,14541.79,0.001093
2,14636.24,14697.25,14516.30,14525.91,-0.008928
3,14670.56,14716.95,14550.88,14656.18,-0.009343
4,14771.53,14794.68,14610.02,14793.76,-0.001968
...,...,...,...,...,...
248,11516.62,11672.05,11507.46,11665.06,0.017171
249,11370.23,11468.26,11343.04,11466.47,0.007594
250,11449.25,11462.05,11297.53,11379.72,0.005986
251,11258.45,11326.22,11245.45,11311.80,0.004150


# **openpyxl Module**

In [19]:
workbook = load_workbook(filename = '/content/Financial_data.xlsx')

In [20]:
nasdaq_sheet = workbook['Nasdaq']

In [21]:
nasdaq_l = []

for row in range(2, nasdaq_sheet.max_row+1):
  nasdaq_l.append(nasdaq_sheet.cell(row, nasdaq_sheet.max_column-1).value)

In [22]:
pct_chnge = []

for row,row_lag in zip(nasdaq_l, nasdaq_l[1:]):
  pct_chnge.append((np.log(row/row_lag)))

pct_chnge.insert(0, 'Log Return')
pct_chnge.append(None)

In [23]:
max_col = nasdaq_sheet.max_column + 1

for row, value in enumerate(pct_chnge, start=1):
    nasdaq_sheet.cell(row=row, column=max_col, value=value)

In [24]:
workbook.save('nasdaq_returns.xlsx')

In [25]:
pd.read_excel('/content/nasdaq_returns.xlsx', sheet_name='Nasdaq')

,Date,NASDAQ Open,NASDAQ High,NASDAQ Low,NASDAQ Close,Log Return
0,8/20/2021,4410.56,4444.35,4406.80,4441.67,0.008905
1,8/19/2021,4382.44,4418.61,4367.73,4405.80,-0.006813
2,8/18/2021,4440.94,4454.32,4397.59,4400.27,-0.004592
3,8/17/2021,4462.12,4462.12,4417.83,4448.08,-0.004479
4,8/16/2021,4461.65,4480.26,4437.66,4479.71,-0.005205
...,...,...,...,...,...,...
248,8/26/2020,3449.97,3481.07,3444.15,3478.73,0.005330
249,8/25/2020,3435.95,3444.21,3425.84,3443.62,0.003717
250,8/24/2020,3418.09,3432.09,3413.13,3431.28,0.009958
251,8/21/2020,3386.01,3399.96,3379.31,3397.16,0.007312


# **References and Additional Learning**

## **Websites**

- **[pandas module Documentation](https://pandas.pydata.org/docs/)**

- **[pyexcel module Documentation](https://docs.pyexcel.org/en/latest/)**

- **[openpyxl module Documentation](https://openpyxl.readthedocs.io/en/stable/)**

# **Connect**
- **Feel free to connect with Adrian on [YouTube](https://www.youtube.com/channel/UCPuDxI3xb_ryUUMfkm0jsRA), [LinkedIn](https://www.linkedin.com/in/adrian-dolinay-frm-96a289106/), [Twitter](https://twitter.com/DolinayG) and [GitHub](https://github.com/ad17171717). Happy coding!**